# Debug continuous image decoding
I noticed that the results of continuous image decoding looked odd, even though the ROI decoding looked good.

Patterns I noticed from the results below:
- pFgA, pFgA_given_pF=0.50, pAgF, and pAgF_given_pF=0.50 all return the most common terms (e.g., task, using, magnetic).
- consistency_z also gives common terms, although a different set (e.g., motor, cerebellum, temporal).
    - The bottom terms seem relevant for faces, emotion, and reward, but not visual.
- specificity_z is a mixed bag:
    - faces has relevant terms at its *lowest* weights
    - emotion has some relevant terms (amygdala is highest; cingulate, negative) at highest weights
    - reward has relevant terms at highest weights
    - so does visual.

In [1]:
from os import mkdir
from os.path import join, isdir
from shutil import rmtree

import gclda
import neurosynth
import pandas as pd

In [2]:
# Use the current Neurosynth database (n=11406 studies, ~3k features)
d = '/Users/tsalo/Desktop/ns-dataset/'
d2 = '/Users/tsalo/Documents/tsalo/python_gclda/data/models/Neurosynth2015Filtered2/'
dset = neurosynth.Dataset.load(join(d, 'updated_dataset.pkl'))

# Load GC-LDA model trained on current Neurosynth database
# (1000 iterations, 400 topics, 2 symmetric subregions per topic) 
model = gclda.Model.load(join(d2, 'model_200topics_2015Filtered2.pkl'))
decoder = gclda.Decoder(model)

names = ['faces', 'emotion', 'reward', 'vision']
terms = ['face* | facia*', 'emo*', 'reward*', 'visua* | visio*']

In [3]:
# Run meta-analyses
if isdir('temp'):
    rmtree('temp/')
mkdir('temp')
for i, term in enumerate(terms):
    name = names[i]
    ids = dset.get_studies(expression=term)
    print('{0}: {1} studies'.format(name, len(ids)))
    ma = neurosynth.meta.MetaAnalysis(dset, ids)
    ma.save_results('temp', name)

Generating LALR tables


faces: 1180 studies


/Users/tsalo/Documents/tsalo/neurosynth/neurosynth/analysis/meta.py:134: RuntimeWarning: invalid value encountered in divide
  pFgA = pAgF * pF / pA
/Users/tsalo/Documents/tsalo/neurosynth/neurosynth/analysis/meta.py:139: RuntimeWarning: invalid value encountered in divide
  pFgA_prior = pAgF * prior / pAgF_prior
Generating LALR tables


emotion: 1690 studies


Generating LALR tables


reward: 719 studies


Generating LALR tables


vision: 2719 studies


## Decode reverse inference z-values

In [4]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_specificity_z.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
            faces    Weight    emotion    Weight     reward    Weight  \
0           motor  0.169276  emotional  0.126189  emotional  0.239309   
1         spatial  0.039770     social  0.049861     social  0.098793   
2       attention  0.035302    emotion  0.049324    outcome  0.073987   
3       movements  0.034377     memory  0.034108     reward  0.073498   
4        auditory  0.032522      faces  0.029890    emotion  0.067346   
5        movement  0.031870  affective  0.023894     values  0.062130   
6          action  0.030883       self  0.022042      value  0.057607   
7  working_memory  0.030445     values  0.021368       gain  0.056008   
8        learning  0.029339       bias  0.019156  affective  0.053861   
9   somatosensory  0.029182       loss  0.018951    arousal  0.052624   

       vision    Weight  
0      visual  0.219567  
1        body  0.042339  
2   attention  0.042328  
3        face  0.041961  
4      motion  0.036513  
5    encoding  0.032529  
6  

## Decode forward inference z-values

In [5]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_consistency_z.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
          faces     Weight       emotion    Weight        reward    Weight  \
0         motor  18.948401         motor  4.293515         motor  6.282046   
1       lobules   4.764082       lobules  1.088736       lobules  1.873589   
2        timing   4.226164        timing  0.954607        timing  1.697116   
3        social   4.147535        lesion  0.803870        visual  1.476396   
4        lesion   3.499482            iv  0.753190       reading  1.393691   
5  sensorimotor   3.467321     asymmetry  0.716585        lesion  1.310424   
6            iv   3.257850         focal  0.710107            iv  1.298904   
7     asymmetry   3.186450  sensorimotor  0.692990  sensorimotor  1.242575   
8         focal   3.183072        verbal  0.674711     asymmetry  1.184920   
9     intensity   3.017914       reading  0.617181         focal  1.181768   

        vision     Weight  
0       visual  23.299077  
1    attention   7.251309  
2         body   6.761068  
3         face   5

## Decode reverse inference conditional probabilities

In [6]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pFgA.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
       faces    Weight    emotion    Weight     reward    Weight     vision  \
0     visual  0.013905  emotional  0.010842  emotional  0.012222     visual   
1  emotional  0.009126     visual  0.010035      motor  0.009626      motor   
2      motor  0.008912      motor  0.009332     visual  0.009566     memory   
3     memory  0.007669     social  0.007344     social  0.007921  emotional   
4     social  0.005945     memory  0.006519     memory  0.005620  attention   
5       face  0.005433   encoding  0.003638       self  0.003791     social   
6   encoding  0.004789   learning  0.003623  retrieval  0.003599    spatial   
7      faces  0.004779       self  0.003590   learning  0.003598   encoding   
8   semantic  0.003561      faces  0.003398    emotion  0.003313       face   
9  retrieval  0.003312  retrieval  0.003363     reward  0.003238   learning   

     Weight  
0  0.017497  
1  0.012203  
2  0.006111  
3  0.005825  
4  0.004885  
5  0.004712  
6  0.004556  
7  0.00

## Decode forward inference conditional probabilities

In [7]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pAgF.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
       faces    Weight    emotion    Weight     reward    Weight     vision  \
0     visual  0.016088     visual  0.011567  emotional  0.011465     visual   
1  emotional  0.009329  emotional  0.011088     visual  0.010618      motor   
2      motor  0.007628      motor  0.007828      motor  0.008347  attention   
3     memory  0.007026     memory  0.006054     social  0.006264     memory   
4       face  0.006876     social  0.005912     memory  0.005043  emotional   
5      faces  0.006172    emotion  0.004628  attention  0.004198    spatial   
6      words  0.004927      faces  0.004271    emotion  0.004123       face   
7     social  0.004669  attention  0.003927   learning  0.003397   auditory   
8   encoding  0.004524   auditory  0.003926       self  0.003304   encoding   
9  attention  0.004405   learning  0.003479   feedback  0.003301      words   

     Weight  
0  0.019457  
1  0.010423  
2  0.006207  
3  0.005404  
4  0.005133  
5  0.004650  
6  0.004522  
7  0.00

## Decode reverse inference posterior probabilities (with uniform prior)

In [8]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pFgA_given_pF=0.50.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
       faces    Weight    emotion    Weight     reward    Weight       vision  \
0     visual  0.013015     visual  0.010874     visual  0.010963       visual   
1      motor  0.010225      motor  0.010492      motor  0.010963        motor   
2  emotional  0.008325  emotional  0.009102  emotional  0.009483    emotional   
3     memory  0.006695     social  0.006624     social  0.006782       memory   
4     social  0.005941     memory  0.006120     memory  0.005730       social   
5   encoding  0.004178   learning  0.003714   learning  0.003787    attention   
6       face  0.003764   encoding  0.003591  attention  0.003527      spatial   
7      faces  0.003551       self  0.003323       self  0.003460     encoding   
8  attention  0.003517  attention  0.003249   encoding  0.003416     learning   
9   learning  0.003516    spatial  0.003233    spatial  0.003352  stimulation   

     Weight  
0  0.015094  
1  0.012381  
2  0.006324  
3  0.005913  
4  0.005058  
5  0.004457  

## Decode forward inference posterior probabilities (with uniform prior)

In [9]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pAgF_given_pF=0.50.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
       faces    Weight    emotion    Weight     reward    Weight     vision  \
0     visual  0.014435     visual  0.012491     visual  0.011959     visual   
1      motor  0.009257      motor  0.009398      motor  0.009497      motor   
2  emotional  0.007857  emotional  0.008539  emotional  0.008952  emotional   
3     memory  0.005883     memory  0.005437     social  0.005364  attention   
4  attention  0.004792     social  0.005125     memory  0.005018     memory   
5       face  0.004765  attention  0.004607  attention  0.004671     social   
6     social  0.004609   auditory  0.004149   auditory  0.003578   auditory   
7      faces  0.004451      faces  0.003598   learning  0.003540    spatial   
8      words  0.004000   learning  0.003589    spatial  0.003444       face   
9   auditory  0.003993    spatial  0.003512    emotion  0.003324   learning   

     Weight  
0  0.015385  
1  0.010486  
2  0.006156  
3  0.005487  
4  0.005141  
5  0.004246  
6  0.004219  
7  0.00

In [10]:
# Perform cleanup
rmtree('temp/')